In [1]:
!pip install rectools[lightfm] -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 7.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 85.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 13.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [1]:
import pandas as pd
import polars as pl
import numpy as np
from catboost import CatBoostRanker, Pool

In [53]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15592152 entries, 0 to 15592151
Data columns (total 13 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   community_id       object 
 2   description        object 
 3   customers_count    int64  
 4   messages_count     int64  
 5   type               int64  
 6   region_id          float64
 7   themeid            float64
 8   business_category  object 
 9   business_parent    object 
 10  customer_id        object 
 11  status             object 
 12  join_request_date  float64
dtypes: float64(3), int64(4), object(6)
memory usage: 1.5+ GB


In [2]:
data = pl.read_csv('/kaggle/input/ok-rec-data/train_df.tsv', separator='\t')

In [3]:
data = data.to_pandas()

In [3]:
data = pd.read_csv('/kaggle/input/ok-rec-data/train_df.tsv', delimiter='\t')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [50]:
len(set(data['customer_id']) & set(test['customer_id']))

69046

In [3]:
train.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


Unnamed: 0                                       community_id  \
0           0  00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...   
1           1  06107f1eae304c45d3e6324cc86f7d39662793a53b159b...   
2           2  06107f1eae304c45d3e6324cc86f7d39662793a53b159b...   
3           3  06107f1eae304c45d3e6324cc86f7d39662793a53b159b...   
4           4  097e4dae68029ecc3c6183ed4386aff71dcac227cbaff7...   

                                         description  customers_count  \
0  Ванга сказала: "Выживет пчела - выживет челове...             2966   
1                                                NaN             8982   
2                                                NaN             8982   
3                                                NaN             8982   
4  Читать внимательно!!! Причины попадания в чёрн...             1035   

   messages_count  type     region_id  themeid business_category  \
0               1     7           NaN      NaN               NaN   
1               2     7  1.042403e+10    246.0               NaN   
2               2     7  1.042403e+10    246.0               NaN   
3               2     7  1.042403e+10    246.0               NaN   
4               1     7  1.041779e+10      NaN          FAN_CLUB   

  business_parent                                        customer_id status  \
0             NaN  947224211267aefcc2e3e9c524fdf46ce329bc638e8bf1...      A   
1             NaN  b935c3390d82df612de19591d2dd16e1eff8e660746bb3...      A   
2             NaN  ff873dee9234ee290d92eb6c15d7179aabf0653dbed3b3...      A   
3             NaN  737ef0e849feda6cdfd350d19fcedaf17aebb757c0a1fc...      A   
4            BLOG  780fb8c178af972826ac0f84e42ac4c6c1dcdeeec0c4dd...      A   

   join_request_date  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN

In [4]:
train['status'].value_counts()

status
A    11337886
P     2604832
I     1619039
B       26367
Y        2077
R        1184
F         767
Name: count, dtype: int64

In [4]:
train = data.copy()

In [ ]:
status2target = {
    'A': 1,
    'P': 0.15,
    'B': -1,
    'Y': 0.5,
    'I': 0.3,
    'R': 0.35,
    'F': 0.55,
}

train['target'] = train['status'].replace(status2target)

/tmp/ipykernel_183/219026227.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['target'] = train['status'].replace(status2target)


In [7]:
import rectools
from rectools.dataset import Dataset
from rectools.dataset.features import DenseFeatures, SparseFeatures, IdMap
from rectools import Columns

In [8]:
train = train[['community_id', 'customer_id', 'target']]

In [9]:
train.columns = [Columns.Item, Columns.User, Columns.Weight]

In [10]:
train[Columns.Datetime] = 1

/tmp/ipykernel_183/4003746283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[Columns.Datetime] = 1


In [11]:
group_features = [
        'community_id', 'description', 'customers_count', 
        'messages_count', 'type', 'region_id', 'themeid',
        'business_category', 'business_parent'
    ]
group_df = data[group_features].drop_duplicates(subset='community_id')

In [12]:
cat_cols_group = ['business_parent']

In [13]:
group_df[cat_cols_group] = group_df[cat_cols_group].astype(object)

In [124]:
group_df

item_id  \
0         00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...   
1         06107f1eae304c45d3e6324cc86f7d39662793a53b159b...   
4         097e4dae68029ecc3c6183ed4386aff71dcac227cbaff7...   
5         0c80a637ed172b520c326ddb1976da403174a308c21c05...   
6         14fb095e366c5692aeb4d1aa4cce555dbc57624fd0a3f9...   
...                                                     ...   
15585934  48cc499c4c693029b6118c2d756591566417222c2f8d5b...   
15587787  6daa92382e1d92e18c2b55605e4d6b58fa7ffa7d875106...   
15587902  833349f8236a920540623b1115fa019be610f68729a9c5...   
15590003  8afe04f7112479b29c246c0c6870754389a64e07ec41f4...   
15590070  931b9f52f9c6a938f0dc7f7a48863efa4f5f5a33641369...   

                                                description  customers_count  \
0         Ванга сказала: "Выживет пчела - выживет челове...             2966   
1                                                       NaN             8982   
4         Читать внимательно!!! Причины попадания в чёрн...             1035   
5         Здесь вы сможете увидеть все спортивные новост...             6573   
6         Все про охоту,природу и рыбалку в  Сибири и за...             3656   
...                                                     ...              ...   
15585934  🌺"Одна из самых больших удач в жизни человека ...              114   
15587787  1 сентября исполняется 35 лет Александровскому...              398   
15587902                                                NaN              150   
15590003                                                NaN              131   
15590070  На Земле много удивительных мест, от которых з...               89   

          messages_count type      region_id themeid   business_category  \
0                      1    7            NaN     NaN                 NaN   
1                      2    7  10424034685.0   246.0                 NaN   
4                      1    7  10417791639.0     NaN            FAN_CLUB   
5                      2    7  42753139041.0   209.0                 NaN   
6                      1    7  10407593565.0   320.0                 NaN   
...                  ...  ...            ...     ...                 ...   
15585934               2    7            NaN     NaN    GOS_KINDERGARTEN   
15587787               2    7  10407113578.0     NaN                 NaN   
15587902               2    7            NaN     NaN    GOS_KINDERGARTEN   
15590003               1    7            NaN     NaN                 NaN   
15590070               1    7            NaN     NaN  TRAVEL_AND_TOURISM   

            business_parent  
0                       NaN  
1                       NaN  
4                      BLOG  
5                       NaN  
6                       NaN  
...                     ...  
15585934  GOS_ORGANIZATIONS  
15587787                NaN  
15587902  GOS_ORGANIZATIONS  
15590003                NaN  
15590070            TOURISM  

[149114 rows x 9 columns]

In [14]:
gf = cat_cols_group

In [15]:
cols = group_df.columns.values
cols[0] = Columns.Item
group_df.columns = cols

In [16]:
item_features_frames = []
for feature in gf:
    feature_frame = group_df.reindex(columns=[Columns.Item, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    item_features_frames.append(feature_frame)
item_features = pd.concat(item_features_frames)

In [17]:
item_features

id  \
0         00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...   
1         06107f1eae304c45d3e6324cc86f7d39662793a53b159b...   
4         097e4dae68029ecc3c6183ed4386aff71dcac227cbaff7...   
5         0c80a637ed172b520c326ddb1976da403174a308c21c05...   
6         14fb095e366c5692aeb4d1aa4cce555dbc57624fd0a3f9...   
...                                                     ...   
15585934  48cc499c4c693029b6118c2d756591566417222c2f8d5b...   
15587787  6daa92382e1d92e18c2b55605e4d6b58fa7ffa7d875106...   
15587902  833349f8236a920540623b1115fa019be610f68729a9c5...   
15590003  8afe04f7112479b29c246c0c6870754389a64e07ec41f4...   
15590070  931b9f52f9c6a938f0dc7f7a48863efa4f5f5a33641369...   

                      value          feature  
0                      None  business_parent  
1                      None  business_parent  
4                      BLOG  business_parent  
5                      None  business_parent  
6                      None  business_parent  
...                     ...              ...  
15585934  GOS_ORGANIZATIONS  business_parent  
15587787               None  business_parent  
15587902  GOS_ORGANIZATIONS  business_parent  
15590003               None  business_parent  
15590070            TOURISM  business_parent  

[149114 rows x 3 columns]

In [117]:
item_features['id'].value_counts()

id
00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc751508acd1d8603d5e9e2ac7c4bc5937ddf8722d3bef535ef624c900b8eef59e077316d4ce292f2a7cd4    7
6045d6e2c67cf8ac8c592b516c21e4702006892a30a9dbabe60f812c4759f39a4a62c024f66714eec1011af5fd95512ddacac3c7893ddaee15d327791b992b67    7
9bdd9ea9fe2f2c424da9e9be0f760915f8509dc6c18398953e6bef8a587806b636e7578b26823414a676ecced80c2c03f1ce46dbca4383f8ef7d3f583cfe3ae5    7
9f33949659b95dd7ba57078346bb4a52875af3dd1e443cb3d766dbdee755f17b3bcc1566ae4d84db774f168ce57e1d975b2469842c8786870de16c2e68c5f854    7
a6f7b38bb75f22a6dcc395f9f7433bab809649d630e90e7c0bb11c2c092b2ce3d5177917e325f265eb6a75eb52d133299090c1ccdc4c3c408817bc84809027f9    7
                                                                                                                                   ..
8b66a6ed69000ead0478f1ebc8996bad85d0e50a5e25dc362d58e5825c48d01595c459f8ed3ca7fe8979f293988d65bbba44c0d4b475afb85238b44eea2ddaa9    7
9cecadd84789218371f7afa74a9e60d54cac244bec81b5d6890fa342cde

In [14]:
ds = Dataset.construct(train)

In [118]:
group_df.select_dtypes(object).columns

Index(['item_id', 'description', 'business_category', 'business_parent'], dtype='object')

In [18]:
ds = Dataset.construct(
    interactions_df=train,
    item_features_df=item_features,
    cat_item_features=cat_cols_group,
)

In [19]:
from rectools.metrics import MAP, calc_metrics, MeanInvUserFreq, Serendipity
from rectools.models import ImplicitALSWrapperModel, PopularModel
from rectools import Columns
from rectools.dataset import Dataset
from implicit.als import AlternatingLeastSquares

In [20]:
metrics_name = {
    'MAP': MAP,
    # 'MIUF': MeanInvUserFreq,
    # 'Serendipity': Serendipity


}
metrics = {}
for metric_name, metric in metrics_name.items():
    for k in (3, 5, 7):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [21]:
K_RECOS = 7
NUM_THREADS = 32
RANDOM_STATE = 32
ITERATIONS = 15

In [13]:
from sklearn.model_selection import train_test_split

tr, val = train_test_split(train['user_id'], train_size=0.8, random_state=0)

In [22]:
def make_base_model(factors: int, regularization: float, alpha: float, fit_features_together: bool=False):
    return ImplicitALSWrapperModel(
        AlternatingLeastSquares(
            factors=factors,
            regularization=regularization,
            alpha=alpha,
            random_state=RANDOM_STATE,
            use_gpu=False,
            num_threads = NUM_THREADS,
            iterations=ITERATIONS),
        fit_features_together = fit_features_together, verbose=1
        )

In [24]:
model = make_base_model(64, 0.1, 10, True) #32

In [18]:
from rectools.models import LightFMWrapperModel
from lightfm import LightFM

model = LightFMWrapperModel(
        # внутри модели указываем параметр no_components
        # это размезность эмбеддингов, которые выучит модель
        model=LightFM(no_components = 64, item_alpha=10)
        , verbose=1, epochs=10, num_threads=NUM_THREADS)

In [96]:
model = PopularModel(verbose=1, popularity='sum_weight')

In [25]:
model.fit(ds)

/usr/local/lib/python3.11/dist-packages/rectools/dataset/features.py:424: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


  0%|          | 0/15 [00:00<?, ?it/s]

In [26]:
test = pd.read_csv('/kaggle/input/ok-rec-data/test_customer_ids.csv')

In [27]:
test_users = test['customer_id'].values

In [28]:
recos = model.recommend(
                users=test_users,
                dataset=ds,
                k=K_RECOS,
                filter_viewed=True,
            )

In [29]:
sub = pd.read_csv('/kaggle/input/ok-rec-data/sample_submission.csv')

In [100]:
recos

user_id  \
0       d811a09d435ac2d1b1ed46e272405af10933b4711f4564...   
1       d811a09d435ac2d1b1ed46e272405af10933b4711f4564...   
2       d811a09d435ac2d1b1ed46e272405af10933b4711f4564...   
3       d811a09d435ac2d1b1ed46e272405af10933b4711f4564...   
4       d811a09d435ac2d1b1ed46e272405af10933b4711f4564...   
...                                                   ...   
483317  f57918286b0047d2884e72631b65b4c7f13f5ffc697743...   
483318  f57918286b0047d2884e72631b65b4c7f13f5ffc697743...   
483319  f57918286b0047d2884e72631b65b4c7f13f5ffc697743...   
483320  f57918286b0047d2884e72631b65b4c7f13f5ffc697743...   
483321  f57918286b0047d2884e72631b65b4c7f13f5ffc697743...   

                                                  item_id         score  rank  
0       56e047468d547344d679664503bd7a882e0bdc7c6cfdd5...  46926.851562     1  
1       4f959232daeac03dcaa0d00f78e5d6eeb79f77a5c091d1...  40012.148438     2  
2       2f20282063ceeb339a5c18ac1ab12b7334cd2482f59cc6...  39567.851562     3  
3       541f065a4a44536ba388a69fcca5351f1727c2484a5d60...  34291.898438     4  
4       fbe92818224f039538512532cab1664bacf89c8de17f0e...  26971.199219     5  
...                                                   ...           ...   ...  
483317  2f20282063ceeb339a5c18ac1ab12b7334cd2482f59cc6...  39567.851562     3  
483318  541f065a4a44536ba388a69fcca5351f1727c2484a5d60...  34291.898438     4  
483319  fbe92818224f039538512532cab1664bacf89c8de17f0e...  26971.199219     5  
483320  a56ece5b11e1e4b4044f1251efe4759cf30d829cbf483f...  24548.199219     6  
483321  0cfae246abc27623203a9abd89e22ee258e956bd9a1d75...  24121.900391     7  

[483322 rows x 4 columns]

In [30]:
res = recos.pivot(
    index='user_id',
    columns='rank',
    values='item_id'
)

In [23]:
res

rank                                                                                                1  \
user_id                                                                                                 
00005f5e3d17a1458728b9d17dfbc5167ecb33696b12533...  cd1602236a70299be13e1fd7d9ca93ddbf61e4b2ba93b5...   
0000db63670898d671ed7f4799760e3090cd25e2c964e8f...  e4dde1f1840767f6978b690c270ee52ec43bac5bd45083...   
0003269c6496abc1a3855b9f0450ab6e83ebc7a42c6565a...  1745a169e492f52eadbbd4e36c04cea05bdd074b3a4a0d...   
00049ae309d9ba1b1d0da48616964c3b5720507a851762c...  83339471c2dc165058970a878ad0d47fde631340d32a5f...   
0004ac231f7eaacfbfa3a5c3c834c2551e8da61f638c162...  c8e93cb170f8a7abf9ae86962ea6f8e9ca410596d7e617...   
...                                                                                               ...   
fff7aac354fe6177e8fc140472ad98bb12031ac2a58a6bd...  ffb65063c64dfb56d9db0c407a34a67bb162dd2555edee...   
fff8dc05f8e7d6e40a1f0ff791bb38f17b9ba7224957913...  30d18489f55297c10e9ba474d0df3a6fd25395fa76748c...   
fff8f5d441bcab65fca89e7ee517de065701d515466dcac...  28c04cdcd271eeb711a9ab0d24e56b970ec41ac9590e8e...   
fffef4d379ef16ff73693dbf265134c93d05cc8f531a55e...  4874ae42655f780b15cc9a4cc98e40153848c193a83858...   
ffffd1f7440cf8497614e2e799a9e61dd883e8b0ccf0ad6...  1745a169e492f52eadbbd4e36c04cea05bdd074b3a4a0d...   

rank                                                                                                2  \
user_id                                                                                                 
00005f5e3d17a1458728b9d17dfbc5167ecb33696b12533...  3cb09f5da8dffc120cc3528a4ba9e2c461c32549042f1b...   
0000db63670898d671ed7f4799760e3090cd25e2c964e8f...  c9bebf3536626f440242a1960ef0ce2f1c4550d592dd98...   
0003269c6496abc1a3855b9f0450ab6e83ebc7a42c6565a...  8da0d3596ccc45c07ca6363561dff9b705b5e52ab6e223...   
00049ae309d9ba1b1d0da48616964c3b5720507a851762c...  e47c4175c63c5111463c0b9aba89e45fb6f50c90c39bf1...   
0004ac231f7eaacfbfa3a5c3c834c2551e8da61f638c162...  f02e0ca92b8b94af79e1ee1f65e54733d2313ad7966eae...   
...                                                                                               ...   
fff7aac354fe6177e8fc140472ad98bb12031ac2a58a6bd...  4874ae42655f780b15cc9a4cc98e40153848c193a83858...   
fff8dc05f8e7d6e40a1f0ff791bb38f17b9ba7224957913...  f91d11a35612680caf2769b679691f29987c98d6fa1400...   
fff8f5d441bcab65fca89e7ee517de065701d515466dcac...  052d4b7ce095e275ccdc89f94d799ab57c4e37d4e1bb34...   
fffef4d379ef16ff73693dbf265134c93d05cc8f531a55e...  14b41899a2b45f94831fd8bab5fd6c7e45a57de30f06ec...   
ffffd1f7440cf8497614e2e799a9e61dd883e8b0ccf0ad6...  ddbb0ee9f1eff9b9ff830faac9db11f178687e332eb28b...   

rank                                                                                                3  \
user_id                                                                                                 
00005f5e3d17a1458728b9d17dfbc5167ecb33696b12533...  740d018e6a429c73010193b9b2096ed9122245f87fd8d1...   
0000db63670898d671ed7f4799760e3090cd25e2c964e8f...  34281c356759f0f8b00e96cce75adac040f98d39d737b7...   
0003269c6496abc1a3855b9f0450ab6e83ebc7a42c6565a...  ddbb0ee9f1eff9b9ff830faac9db11f178687e332eb28b...   
00049ae309d9ba1b1d0da48616964c3b5720507a851762c...  14b26c6573a27d80243aa81ef4e3110edc3d3b69a0e424...   
0004ac231f7eaacfbfa3a5c3c834c2551e8da61f638c162...  5725a9009fc6ec25a3304547ff235fe1be070780458665...   
...                                                                                               ...   
fff7aac354fe6177e8fc140472ad98bb12031ac2a58a6bd...  0cfae246abc27623203a9abd89e22ee258e956bd9a1d75...   
fff8dc05f8e7d6e40a1f0ff791bb38f17b9ba7224957913...  6faab67486c16231cbb50e43e85b9ca8db9be4af01d7c6...   
fff8f5d441bcab65fca89e7ee517de065701d515466dcac...  8ac53dd1e21ed34aeb4eaa9c6c8a3e46396f43a75401bb...   
fffef4d379ef16ff73693dbf265134c93d05cc8f531a55e...  f146596214865eb9e4abf6a4d494480e4ad79a0191bd25...   
ffffd1f7

In [35]:
res.columns

Index([1, 2, 3, 4, 5, 6, 7], dtype='int64', name='rank')

In [31]:
res = res.reset_index()

In [32]:
res.columns = sub.columns

In [90]:
res

customer_id  \
0      00005f5e3d17a1458728b9d17dfbc5167ecb33696b1253...   
1      0000db63670898d671ed7f4799760e3090cd25e2c964e8...   
2      0003269c6496abc1a3855b9f0450ab6e83ebc7a42c6565...   
3      00049ae309d9ba1b1d0da48616964c3b5720507a851762...   
4      0004ac231f7eaacfbfa3a5c3c834c2551e8da61f638c16...   
...                                                  ...   
69041  fff7aac354fe6177e8fc140472ad98bb12031ac2a58a6b...   
69042  fff8dc05f8e7d6e40a1f0ff791bb38f17b9ba722495791...   
69043  fff8f5d441bcab65fca89e7ee517de065701d515466dca...   
69044  fffef4d379ef16ff73693dbf265134c93d05cc8f531a55...   
69045  ffffd1f7440cf8497614e2e799a9e61dd883e8b0ccf0ad...   

                                          community_id_1  \
0      cd1602236a70299be13e1fd7d9ca93ddbf61e4b2ba93b5...   
1      e4dde1f1840767f6978b690c270ee52ec43bac5bd45083...   
2      1745a169e492f52eadbbd4e36c04cea05bdd074b3a4a0d...   
3      e47c4175c63c5111463c0b9aba89e45fb6f50c90c39bf1...   
4      c8e93cb170f8a7abf9ae86962ea6f8e9ca410596d7e617...   
...                                                  ...   
69041  4874ae42655f780b15cc9a4cc98e40153848c193a83858...   
69042  30d18489f55297c10e9ba474d0df3a6fd25395fa76748c...   
69043  28c04cdcd271eeb711a9ab0d24e56b970ec41ac9590e8e...   
69044  4874ae42655f780b15cc9a4cc98e40153848c193a83858...   
69045  1745a169e492f52eadbbd4e36c04cea05bdd074b3a4a0d...   

                                          community_id_2  \
0      3cb09f5da8dffc120cc3528a4ba9e2c461c32549042f1b...   
1      c9bebf3536626f440242a1960ef0ce2f1c4550d592dd98...   
2      8da0d3596ccc45c07ca6363561dff9b705b5e52ab6e223...   
3      83339471c2dc165058970a878ad0d47fde631340d32a5f...   
4      802490211a7672b9b79d2da28e981aa4b49bdfc428c929...   
...                                                  ...   
69041  ffb65063c64dfb56d9db0c407a34a67bb162dd2555edee...   
69042  f91d11a35612680caf2769b679691f29987c98d6fa1400...   
69043  3af949566fdaacafca98193fe96e5a4e860518ae143d14...   
69044  14b41899a2b45f94831fd8bab5fd6c7e45a57de30f06ec...   
69045  ddbb0ee9f1eff9b9ff830faac9db11f178687e332eb28b...   

                                          community_id_3  \
0      75d9d8710b5d2a6ef32c420b0292bfc87dcbf1476fb01b...   
1      34281c356759f0f8b00e96cce75adac040f98d39d737b7...   
2      ddbb0ee9f1eff9b9ff830faac9db11f178687e332eb28b...   
3      7c083e85ef56f11eec1af74b47aa5c52adae2d232f1a6c...   
4      426b44e084c07cbab8b83fc88016216909f3a577016757...   
...                                                  ...   
69041  f146596214865eb9e4abf6a4d494480e4ad79a0191bd25...   
69042  6faab67486c16231cbb50e43e85b9ca8db9be4af01d7c6...   
69043  052d4b7ce095e275ccdc89f94d799ab57c4e37d4e1bb34...   
69044  f146596214865eb9e4abf6a4d494480e4ad79a0191bd25...   
69045  8da0d3596ccc45c07ca6363561dff9b705b5e52ab6e223...   

                                          community_id_4  \
0      740d018e6a429c73010193b9b2096ed9122245f87fd8d1...   
1      80827b903e46210d7bf8ba5a65ea2a3d5a5ab894dc08bd...   
2      02cd6559d4b4dd60e62a9f8c9b9a40a70d521dd7e88ee8...   
3      14b26c6573a27d80243aa81ef4e3110edc3d3b69a0e424...   
4      f02e0ca92b8b94af79e1ee1f65e54733d2313ad7966eae...   
...                                                  ...   
69041  0cfae246abc27623203a9abd89e22ee258e956bd9a1d75...   
69042  e76e0111299dd28f137ad9df02142d711d3267cb7420fd...   
69043  8ac53dd1e21ed34aeb4eaa9c6c8a3e46396f43a75401bb...   
69044  1eedb3ec54c472f73922b16f91cee127297ce7ddfb606e...   
69045  3cb09f5da8dffc120cc3528a4ba9e2c461c32549042f1b...   

                                          community_id_5  \
0      3c7f194ee5620cd8b719a2cbbd7a97f5b6142fa85750bf...   
1      a54a4c95b7a5cb88074c00a120247d8ca48799bec4b31b...   
2      0bddb3c8454a3d48e6e6b4d120ed4c65cf8d5c885a8b65...   
3      f4b60a62b1630c04f96c7d9a534b9a6080200a0a2d87aa...   
4      5725a9009fc6ec25a3304547ff235fe1be070780458665...   
...                                                  ...   
69041  14b41899a2b45

In [33]:
sub = sub[['customer_id']].merge(res, on='customer_id', how='left')

In [27]:
sub['community_id_3'].value_counts()

community_id_3
ddbb0ee9f1eff9b9ff830faac9db11f178687e332eb28b8eb4dfe61e7487a2b2d29ceaaeccd0aefd91681b541d14e138e51c8ec6ccb989bea97c8d1609ec74b5    1042
8da0d3596ccc45c07ca6363561dff9b705b5e52ab6e2235d261fd25a53b572d9a4f14b26c03426e0af622a2e53311787a9ec14e2ee93928862ed917faaebcffa     846
0bddb3c8454a3d48e6e6b4d120ed4c65cf8d5c885a8b65bd70ad638afb36e999cc9274aef17e75220cd766683ee1de2cc4edcb5cf6cc657895e9780a2af6e616     769
3823a5e13dfd871d3ff72d5a69af9b77466d604f43cecdf528e48de66dcf613b883d6e8117916a4eabd079f82f01bb722146ae0cc99895ee5b120cd913c4bb88     697
e79330e7bcea7c9e2c4ecdd4a837db214bf7b097d9fca4f47cd2a8d93a0b5408d4b9f0f178eabee3e990e98f4b7d36b23c207a1b574fa17b351916ae05569e66     695
                                                                                                                                    ... 
5289f7b711cfa257eda398cf0f4cc878597eb57a50e68b6869cbdd8027ecb107d9c8f9e52a7c2f6125f3a00112469b978a6efe958b62e5f18d82592c2f96201f       1
d4069b362c6971caaf55768b93

In [34]:
sub.to_csv('sub_ok_als64_15it_buisness_par.csv', index=False)

In [35]:
import zipfile

with zipfile.ZipFile('sub_ok_als64_15it_buisness_par.zip', 'w') as zipf:
        zipf.write('sub_ok_als64_15it_buisness_par.csv', arcname='sub_ok_als64_15it_buisness_par.csv')

In [10]:
from rectools.models import LightFMWrapperModel
from lightfm import LightFM

model = LightFMWrapperModel(
        # внутри модели указываем параметр no_components
        # это размезность эмбеддингов, которые выучит модель
        model=LightFM(no_components = 10, item_alpha=1)
        , verbose=1)

In [ ]:
model.fit(ds)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
train['target'].value_counts(dropna=False)

target
 1.00    11337886
 0.15     2604832
 0.30     1619039
-1.00       26367
 0.50        2077
 0.35        1184
 0.55         767
Name: count, dtype: int64

In [14]:
train['customer_id'].value_counts()

customer_id
edfb0178c159a127d5305fc277f17c893a95ab401128b222c58532439c1ea25beb449f14d182d111b674b7c5d68e7fdbfb1b1d5311f5312c7d84fe50fc9834a4    18688
456fcb04a6c81a0e43136098083058dbd2156972542ee92b8e12dd7077fab848a53f61479c1a39036d94bf63625da119b09fe74514825ef7978e2a1ac756582f    18308
6e5863120f6fd0cd6d38eb193af8a9eaa6c181e9fbd164e9cd2e45e9d2fc76c6fe2bb168b05b78f6b72c9eab4c957d1f45d03929154751325c7679f69647dcf1    14744
8946a629ab4d16f56a5f16097a4fa85f7853b25e448127b59a7b9819165a94203e463a80aa0eef97be150244884e106638ae4b206e7a04bdc496611058b532b7    13716
4393f79d63b5bfdb42132d4361148ff220c151c1bdf67ef123bda9893d1b78dbfda80dafc5ed6b74fd363320445267e6863674a4ceb77511472b5742d967e6c5    12540
                                                                                                                                    ...  
4bed579a78860253b18d17e035a319ad8309b6144943ddded456956eff9baa8d0a8d941f2e138c6590dae1e46d1e3b0e9187285682559c9b03527f6c6027dbda        1
244d919073597c66b0bf39

In [20]:
test = pd.read_csv('/kaggle/input/ok-rec-data/test_customer_ids.csv')

In [16]:
test['customer_id'].value_counts()

customer_id
d811a09d435ac2d1b1ed46e272405af10933b4711f4564a2a34d9fbe4babc260733cae5f595bad6f0f92e63ac799b5dd97a2355929081ee78061e7f5117451bb    1
6ff8fdf74d6177ad333309e86090e533e6eb03efcb7c59b789132dcd3dde8d198ef5a505fe12a773a86c55f6e352a0eb9a9611fa4c75664f65aaaf839d262ca0    1
a7f59ffb3205d9b3c14c6b8c911839c7a7d28e1afcc2a3d71304ad4df9f4aaf9cdfe989903c8983655b4be75f265e728409523b44ebe4f953d70986e7d497010    1
0bcb31526c82572c414410b53df0e7e91b58b57c7f5d0b9e001b84ec509d4e2e9d902f7c4dfcb707bad0c99367e42be5f65a080cf232ccabfc88ca087af0fc08    1
e3a70febee6396d28301fe313deecc1170504a88fbbf9d97b28a59b739e5a6f2f7b5b7826e0bd6ec310da2258cfe8c65a713b573d34819974f6f90ea0428f1d1    1
                                                                                                                                   ..
96c39521157dadf77c2a6f4ca4683fff5b986b029f2f05abc76583e2206ecc617492ae204994094a26efe701f6f480f1e54a67c0bd9820e81ad4ee568284c408    1
aba45b8b4ad74376e077439460e34f12b05842b64abeeab31e

In [18]:
len(set(test['customer_id'].unique()) & set(train['customer_id'].unique()))

69046

In [19]:
train

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


Unnamed: 0                                       community_id  \
0                  0  00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...   
1                  1  06107f1eae304c45d3e6324cc86f7d39662793a53b159b...   
2                  2  06107f1eae304c45d3e6324cc86f7d39662793a53b159b...   
3                  3  06107f1eae304c45d3e6324cc86f7d39662793a53b159b...   
4                  4  097e4dae68029ecc3c6183ed4386aff71dcac227cbaff7...   
...              ...                                                ...   
15592147    15592147  fd93a38346c35aec0f6b89713206ef55e1bd67604942a5...   
15592148    15592148  fd93a38346c35aec0f6b89713206ef55e1bd67604942a5...   
15592149    15592149  fe59ed819f2e4adef34c1539b1bdb354e066079a86721b...   
15592150    15592150  fe59ed819f2e4adef34c1539b1bdb354e066079a86721b...   
15592151    15592151  fe59ed819f2e4adef34c1539b1bdb354e066079a86721b...   

                                                description  customers_count  \
0         Ванга сказала: "Выживет пчела - выживет челове...             2966   
1                                                       NaN             8982   
2                                                       NaN             8982   
3                                                       NaN             8982   
4         Читать внимательно!!! Причины попадания в чёрн...             1035   
...                                                     ...              ...   
15592147  Чем интересна наша фамилия?\r\nЧем интересны м...              713   
15592148  Чем интересна наша фамилия?\r\nЧем интересны м...              713   
15592149                                                NaN              369   
15592150                                                NaN              369   
15592151                                                NaN              369   

          messages_count  type     region_id  themeid business_category  \
0                      1     7           NaN      NaN               NaN   
1                      2     7  1.042403e+10    246.0               NaN   
2                      2     7  1.042403e+10    246.0               NaN   
3                      2     7  1.042403e+10    246.0               NaN   
4                      1     7  1.041779e+10      NaN          FAN_CLUB   
...                  ...   ...           ...      ...               ...   
15592147               1     7           NaN      NaN               NaN   
15592148               1     7           NaN      NaN               NaN   
15592149               1     7           NaN      NaN               NaN   
15592150               1     7           NaN      NaN               NaN   
15592151               1     7           NaN      NaN               NaN   

         business_parent                                        customer_id  \
0                    NaN  947224211267aefcc2e3e9c524fdf46ce329bc638e8bf1...   
1                    NaN  b935c3390d82df612de19591d2dd16e1eff8e660746bb3...   
2                    NaN  ff873dee9234ee290d92eb6c15d7179aabf0653dbed3b3...   
3                    NaN  737ef0e849feda6cdfd350d19fcedaf17aebb757c0a1fc...   
4                   BLOG  780fb8c178af972826ac0f84e42ac4c6c1dcdeeec0c4dd...   
...                  ...                                                ...   
15592147             NaN  f9b4dfa4e706a0fce61667d53676f80afa7bf22ac42b29...   
15592148             NaN  23bce03052e87ec4a950639335cef0ae14198292b69a9a...   
15592149             NaN  052961072a349e9db6990bb6be0d3b92caedae75436deb...   
15592150             NaN  489ded2f9e283cf68ba0b51a05c43e637e15e6c16265e1...   
15592151             NaN  33c01c29958320959ad15f6c78369211cce26edec4f911...   

         status  join_request_date  target  
0             A                NaN    1.00  
1             A                NaN    1.00  
2             A                NaN    1.00  
3             A                NaN    1.00  
4             A                NaN    1.00  
...         ...             

In [5]:
train_pair = train[['customer_id', 'community_id', 'target']]

In [6]:
train_pair['pair'] = train_pair['customer_id'].astype(str) + '_' + train_pair['community_id'].astype(str)

/tmp/ipykernel_418/1675766382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pair['pair'] = train_pair['customer_id'].astype(str) + '_' + train_pair['community_id'].astype(str)


In [7]:
import random
from tqdm import tqdm
new_data = dict().fromkeys(['customer_id', 'community_id', 'target'], 0)
for x in new_data.keys():
    new_data[x] = []
all_users = train_pair['customer_id'].unique()
all_groups = train_pair['community_id'].unique()
for _ in tqdm(range(len(train_pair))):
    u = all_users[random.randint(0, len(all_users)-1)]
    g = all_groups[random.randint(0, len(all_groups)-1)]
    # if len(train_pair[(train_pair['customer_id'] == u) & (train_pair['community_id'] == g)]) == 0:
    new_data['customer_id'].append(u)
    new_data['community_id'].append(g)
    new_data['target'].append(0)

100%|██████████| 15592152/15592152 [00:42<00:00, 367423.52it/s]


In [8]:
new_data = pd.DataFrame(new_data)
new_data['pair'] = new_data['customer_id'].astype(str) + '_' + new_data['community_id'].astype(str)

In [11]:
new_data

customer_id  \
0         4810f2b76e82ddae6a383cff47f7769ad44d5942ff75f1...   
1         5f7be5706008a5261eafbbcdb744daf76b71bb77390275...   
2         c6c8e8d26530389fe70c656d19186e5a8ceaf834b234f0...   
3         5b3344863f0b28c108d24c5086cfee6c0b59da05a3c801...   
4         1479a2d81ebb6b358d92dbeddfda188753bc0ba9a6759e...   
...                                                     ...   
15592147  c98039cb71f3748477897a38e9f83d6a0418d2048655e2...   
15592148  f8aa66bd7e248e531e9d30eedda3ad59d930e71887203c...   
15592149  8461b1a2665424965e3506b1c17fdaecd16675cf887466...   
15592150  278778ed74e35d5fce64fdb06bb948f5ccc63233f7157e...   
15592151  5bf2617880404d845369a052f5d00b833bae8d602f821a...   

                                               community_id  target  \
0         0ece4dba5fd9c0383a6bfe665c5f597b8eca718089a6e0...       0   
1         cb6367f7e3a7e5d55c7cf731fd7ee01be668f2db466836...       0   
2         c6e6e6f2cb71d0e94848c72dc9f3acd06166690c97cb1d...       0   
3         c995a7fd3d9c30815a8995beb259863ae3575bee53f8dd...       0   
4         f3fe223944aed0b966a8f83a4bb728671817dd4e3214e9...       0   
...                                                     ...     ...   
15592147  005a47173771e51b0ab8c1390ca80fd16d517f843c0785...       0   
15592148  987e4cbb0d3c0b8903e395c7123f0df94fc33886da8b76...       0   
15592149  ccac066b45fdbb89c7faeb626ce6804b4fa21216880a77...       0   
15592150  fe7270b19457dcfc301baf22c4b398e93dcae470cd2068...       0   
15592151  61f6487b3e95cee87e15a7e1b6ae9e269df26fa98ba7c2...       0   

                                                       pair  
0         4810f2b76e82ddae6a383cff47f7769ad44d5942ff75f1...  
1         5f7be5706008a5261eafbbcdb744daf76b71bb77390275...  
2         c6c8e8d26530389fe70c656d19186e5a8ceaf834b234f0...  
3         5b3344863f0b28c108d24c5086cfee6c0b59da05a3c801...  
4         1479a2d81ebb6b358d92dbeddfda188753bc0ba9a6759e...  
...                                                     ...  
15592147  c98039cb71f3748477897a38e9f83d6a0418d2048655e2...  
15592148  f8aa66bd7e248e531e9d30eedda3ad59d930e71887203c...  
15592149  8461b1a2665424965e3506b1c17fdaecd16675cf887466...  
15592150  278778ed74e35d5fce64fdb06bb948f5ccc63233f7157e...  
15592151  5bf2617880404d845369a052f5d00b833bae8d602f821a...  

[15591392 rows x 4 columns]

In [10]:
new_data = new_data[~new_data['pair'].isin(train_pair['pair'])]

In [12]:
train_pair = pd.concat([train_pair, new_data]).sample(frac=1.0, ignore_index=True)

In [13]:
from sklearn.model_selection import train_test_split

train_pair, _ = train_test_split(train_pair, train_size=100_000, stratify=train_pair['target'])

In [17]:
train_pair['target'].value_counts()

target
 1.00    11337886
 0.00     7795692
 0.15     2604832
 0.30     1619039
-1.00       26367
 0.50        2077
 0.35        1184
 0.55         767
Name: count, dtype: int64

In [25]:
def generate_features(df):
    def most_freq(series):
        modes = series.mode()
        return modes.iloc[0] if not modes.empty else None

    # Признаки групп
    group_features = [
        'community_id', 'description', 'customers_count', 
        'messages_count', 'type', 'region_id', 'themeid',
        'business_category', 'business_parent'
    ]
    
    # Признаки пользователей (агрегация по группам)
    user_features = df.groupby('customer_id').agg({
        'community_id': 'count',
        'customers_count': ['mean', 'max', 'min'],
        'messages_count': ['mean', 'max', 'min'],
        'type': most_freq,
        'region_id': most_freq,
        'themeid': most_freq,
        # 'target': ['mean', 'max', 'min', 'std'],
        'business_category': most_freq,
        'business_parent': most_freq,
        'join_request_date': ['mean', 'max', 'min'],
    })
    
    # Выравнивание мультииндекса столбцов
    user_features.columns = ['_'.join(col).strip() for col in user_features.columns.values]
    user_features = user_features.reset_index()
    
    return user_features, group_features

In [26]:
user_f, group_f = generate_features(train)

In [20]:
user_f

customer_id  \
0        000004017025a08c984916e57658b1ab64deb4172db163...   
1        0000079a765e2fd7fbb3d8c6968474d7a40cc9ada4c9ec...   
2        000015dccbb94f6926bfedc1578d6207ab10c6580fb121...   
3        00001a206a3508a172a7be2b19c450b72adfc46239e1ed...   
4        0000208fc1af439a84a92acd34352268ab169b1a6536e6...   
...                                                    ...   
1568441  ffffe958dc1b390cf49fa987bb1b170f73f0e56a78a936...   
1568442  ffffea23f1b7986780b338ea84175e35790ae9b2e3688e...   
1568443  ffffea67894044381d713871f6cb6d482a45c031b1272f...   
1568444  ffffea770d95bb8506956d19721d583365949de286c3fe...   
1568445  fffffe753d5c2d15b43927f71a6a8ab3e63d7137c0bcc0...   

         community_id_count  customers_count_mean  customers_count_max  \
0                        11         115477.545455               279299   
1                        15         132782.533333               741614   
2                         2            948.000000                  948   
3                         1            265.000000                  265   
4                         1         133799.000000               133799   
...                     ...                   ...                  ...   
1568441                   1         115355.000000               115355   
1568442                  11         229143.000000               802708   
1568443                  14         108120.571429               683312   
1568444                   1          36072.000000                36072   
1568445                   1           4223.000000                 4223   

         customers_count_min  messages_count_mean  messages_count_max  \
0                      56227            15.363636                  56   
1                        696            77.800000                 741   
2                        948             5.000000                   5   
3                        265             1.000000                   1   
4                     133799             7.000000                   7   
...                      ...                  ...                 ...   
1568441               115355           548.000000                 548   
1568442                 4116            55.909091                 243   
1568443                 1282             4.357143                  27   
1568444                36072             3.000000                   3   
1568445                 4223             4.000000                   4   

         messages_count_min  type_most_freq  region_id_most_freq  \
0                         2               7         1.039889e+10   
1                         1               7         4.316580e+10   
2                         5               7         5.355300e+04   
3                         1               7                  NaN   
4                         7               7         1.040799e+10   
...                     ...             ...                  ...   
1568441                 548               7                  NaN   
1568442                   1               7         1.041293e+10   
1568443                   1               7         1.040494e+10   
1568444                   3               7                  NaN   
1568445                   4               7         1.041010e+10   

         themeid_most_freq business_category_most_freq  \
0                    716.0         COOKING_AND_RECIPES   
1                    310.0                    CREATION   
2                      NaN       RELIGIOUS_INSTITUTION   
3                    763.0        GOS_HIGHER_EDUCATION   
4                    286.0             CULTURE_AND_ART   
...                    ...                         ...   
1568441              373.0                        None   
1568442               44.0               BEAUTY_SALOON   
1568443               73.0              CARS_MACHINERY   
1568444              290.0                     BLOGGER   
1568445              320.0  INTERIOR_DESIGN_RENOVATION   

        business_paren

In [27]:
group_f = train[group_f].drop_duplicates(subset='community_id')

In [22]:
group_f

community_id  \
0         00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...   
1         06107f1eae304c45d3e6324cc86f7d39662793a53b159b...   
4         097e4dae68029ecc3c6183ed4386aff71dcac227cbaff7...   
5         0c80a637ed172b520c326ddb1976da403174a308c21c05...   
6         14fb095e366c5692aeb4d1aa4cce555dbc57624fd0a3f9...   
...                                                     ...   
15585934  48cc499c4c693029b6118c2d756591566417222c2f8d5b...   
15587787  6daa92382e1d92e18c2b55605e4d6b58fa7ffa7d875106...   
15587902  833349f8236a920540623b1115fa019be610f68729a9c5...   
15590003  8afe04f7112479b29c246c0c6870754389a64e07ec41f4...   
15590070  931b9f52f9c6a938f0dc7f7a48863efa4f5f5a33641369...   

                                                description  customers_count  \
0         Ванга сказала: "Выживет пчела - выживет челове...             2966   
1                                                       NaN             8982   
4         Читать внимательно!!! Причины попадания в чёрн...             1035   
5         Здесь вы сможете увидеть все спортивные новост...             6573   
6         Все про охоту,природу и рыбалку в  Сибири и за...             3656   
...                                                     ...              ...   
15585934  🌺"Одна из самых больших удач в жизни человека ...              114   
15587787  1 сентября исполняется 35 лет Александровскому...              398   
15587902                                                NaN              150   
15590003                                                NaN              131   
15590070  На Земле много удивительных мест, от которых з...               89   

          messages_count  type     region_id  themeid   business_category  \
0                      1     7           NaN      NaN                 NaN   
1                      2     7  1.042403e+10    246.0                 NaN   
4                      1     7  1.041779e+10      NaN            FAN_CLUB   
5                      2     7  4.275314e+10    209.0                 NaN   
6                      1     7  1.040759e+10    320.0                 NaN   
...                  ...   ...           ...      ...                 ...   
15585934               2     7           NaN      NaN    GOS_KINDERGARTEN   
15587787               2     7  1.040711e+10      NaN                 NaN   
15587902               2     7           NaN      NaN    GOS_KINDERGARTEN   
15590003               1     7           NaN      NaN                 NaN   
15590070               1     7           NaN      NaN  TRAVEL_AND_TOURISM   

            business_parent  
0                       NaN  
1                       NaN  
4                      BLOG  
5                       NaN  
6                       NaN  
...                     ...  
15585934  GOS_ORGANIZATIONS  
15587787                NaN  
15587902  GOS_ORGANIZATIONS  
15590003                NaN  
15590070            TOURISM  

[149114 rows x 9 columns]

In [14]:
all_pairs = train[['customer_id', 'community_id']].drop_duplicates()

In [15]:
del train

In [26]:
train_pair['target'].value_counts()

target
 1.00    11337886
 0.00     7795692
 0.15     2604832
 0.30     1619039
-1.00       26367
 0.50        2077
 0.35        1184
 0.55         767
Name: count, dtype: int64

In [29]:
user_f[['region_id_most_freq', 'themeid_most_freq']] = user_f[['region_id_most_freq', 'themeid_most_freq']].astype(object)

In [32]:
group_f[['region_id', 'themeid']] = group_f[['region_id', 'themeid']].astype(object)

In [33]:
group_f.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149114 entries, 0 to 15590070
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   community_id       149114 non-null  object
 1   description        122739 non-null  object
 2   customers_count    149114 non-null  int64 
 3   messages_count     149114 non-null  int64 
 4   type               149114 non-null  int64 
 5   region_id          66747 non-null   object
 6   themeid            48998 non-null   object
 7   business_category  71832 non-null   object
 8   business_parent    71832 non-null   object
dtypes: int64(3), object(6)
memory usage: 11.4+ MB


In [16]:
from sklearn.model_selection import train_test_split

tr_users, val_users = train_test_split(train_pair['customer_id'].unique(), test_size=0.2, random_state=0)

In [17]:
tr_pairs, val_pairs = train_pair[train_pair['customer_id'].isin(tr_users)], train_pair[train_pair['customer_id'].isin(val_users)]

In [28]:
cat_cols_group = group_f.select_dtypes(object).columns.values.tolist()
cat_cols_user = user_f.select_dtypes(object).columns.values.tolist()
cat_cols = list(set(cat_cols_user + cat_cols_group))

In [29]:
cat_cols.remove('description')

In [30]:
user_f[cat_cols_user] = user_f[cat_cols_user].fillna("None")
group_f[cat_cols_group] = group_f[cat_cols_group].fillna("None")

In [48]:
user_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1568446 entries, 0 to 1568445
Data columns (total 16 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   customer_id                  1568446 non-null  object 
 1   community_id_count           1568446 non-null  int64  
 2   customers_count_mean         1568446 non-null  float64
 3   customers_count_max          1568446 non-null  int64  
 4   customers_count_min          1568446 non-null  int64  
 5   messages_count_mean          1568446 non-null  float64
 6   messages_count_max           1568446 non-null  int64  
 7   messages_count_min           1568446 non-null  int64  
 8   type_most_freq               1568446 non-null  int64  
 9   region_id_most_freq          1568446 non-null  object 
 10  themeid_most_freq            1568446 non-null  object 
 11  business_category_most_freq  1568446 non-null  object 
 12  business_parent_most_freq    1568446 non-n

In [18]:
del tr_pairs['pair'], val_pairs['pair']

In [19]:
tr_pairs = tr_pairs.sort_values('customer_id')
val_pairs = val_pairs.sort_values('customer_id')

In [20]:
tr_pool = Pool(data=tr_pairs,
               label=tr_pairs['target'],
               group_id=tr_pairs['customer_id'],
               cat_features=tr_pairs.select_dtypes(object).columns.values)

In [21]:
val_pool = Pool(data=val_pairs,
               label=val_pairs['target'],
               group_id=val_pairs['customer_id'],
               cat_features=val_pairs.select_dtypes(object).columns.values)

In [37]:
tr_pairs.merge(user_f, on='customer_id', how='left').merge(group_f, on='community_id', how='left').drop(columns=['customer_id', 'community_id'])

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

target  community_id_count  customers_count_mean  customers_count_max  \
0         1.0                 153          95989.333333               802708   
1         1.0                 153          95989.333333               802708   
2         0.3                 105          59719.314286               322341   
3         0.0                  18         182826.222222               802708   
4         1.0                 198          95116.803030              2013746   
...       ...                 ...                   ...                  ...   
80084     0.0                  28         608618.000000              2095612   
80085     1.0                   3          25872.000000                44762   
80086     0.0                   6         272104.666667               802708   
80087     1.0                  30         607691.766667              2013746   
80088     0.0                   5         274463.200000               654711   

       customers_count_min  messages_count_mean  messages_count_max  \
0                      149           116.398693                3413   
1                      149           116.398693                3413   
2                      113           561.285714                3218   
3                      525            46.222222                 243   
4                       70            25.050505                 819   
...                    ...                  ...                 ...   
80084                17705            11.357143                  61   
80085                 5402            30.666667                  66   
80086                  538            82.333333                 243   
80087                 2051            45.300000                 270   
80088                20815           108.600000                 503   

       messages_count_min  type_most_freq  region_id_most_freq  ...  \
0                       1               7         1.040799e+10  ...   
1                       1               7         1.040799e+10  ...   
2                       0               7         1.040799e+10  ...   
3                       1               7         1.040799e+10  ...   
4                       0               7         1.040799e+10  ...   
...                   ...             ...                  ...  ...   
80084                   2               7         1.040118e+10  ...   
80085                   5               7         1.039984e+10  ...   
80086                   1               7         1.039910e+10  ...   
80087                   1               7         1.040799e+10  ...   
80088                   1               7         1.039951e+10  ...   

       join_request_date_max join_request_date_min  \
0                        NaN                   NaN   
1                        NaN                   NaN   
2                        NaN                   NaN   
3                        NaN                   NaN   
4                        NaN                   NaN   
...                      ...                   ...   
80084           1.499891e+12          1.457727e+12   
80085           1.552162e+12          1.552162e+12   
80086                    NaN                   NaN   
80087                    NaN                   NaN   
80088                    NaN                   NaN   

                                             description  customers_count  \
0      Я рада приветствовать вас в группе \n" Зеленая...            25524   
1      Добро пожаловать в группу любительниц эксклюзи...           621734   
2      Рецепты. Кулинарные рецепты блюд. Салаты, перв...           101221   
3      Эта группа для тех, кто когда-то учился или ра...              131   
4      — Пригласите в группу всех своих друзей - это ...           290472   
...                                                  ...              ...   
80084  ГРУППА ДЛЯ ОБЩЕНИЯ! НАША ТЕРРИТОРИЯ-НАШИ ПРАВИ...             9326   
80085  АССАЛОМУ АЛЕЙКУМ ХУРМАТЛИ ЮРТДОШЛАР!\nКИМДА ИШ...            44762  

In [38]:
cat_cols.remove('customer_id')
cat_cols.remove('community_id')

In [39]:
tr_pool = Pool(data=tr_pairs.merge(user_f, on='customer_id', how='left').merge(group_f, on='community_id', how='left').drop(columns=['customer_id', 'community_id']),
               label=tr_pairs['target'],
               group_id=tr_pairs['customer_id'],
               cat_features=cat_cols,
               text_features=['description'])

In [40]:
val_pool = Pool(data=val_pairs.merge(user_f, on='customer_id', how='left').merge(group_f, on='community_id', how='left').drop(columns=['customer_id', 'community_id']),
               label=val_pairs['target'],
               group_id=val_pairs['customer_id'],
               cat_features=cat_cols,
               text_features=['description'])

In [35]:
tr_pairs['customer_id'].value_counts()

customer_id
edfb0178c159a127d5305fc277f17c893a95ab401128b222c58532439c1ea25beb449f14d182d111b674b7c5d68e7fdbfb1b1d5311f5312c7d84fe50fc9834a4    65
8946a629ab4d16f56a5f16097a4fa85f7853b25e448127b59a7b9819165a94203e463a80aa0eef97be150244884e106638ae4b206e7a04bdc496611058b532b7    48
456fcb04a6c81a0e43136098083058dbd2156972542ee92b8e12dd7077fab848a53f61479c1a39036d94bf63625da119b09fe74514825ef7978e2a1ac756582f    44
e8c077581788786e83c2f670bfa6ce391187165413c2df021f14e05eee4308d4ce35526f801bb71029eca0045bce8b8e7aa5eeec9d92cf194f466e3b054e7ebc    40
6e5863120f6fd0cd6d38eb193af8a9eaa6c181e9fbd164e9cd2e45e9d2fc76c6fe2bb168b05b78f6b72c9eab4c957d1f45d03929154751325c7679f69647dcf1    38
                                                                                                                                    ..
57f34da2d49e686af48a5dab1e52441e3bdd4691b91a1d234669a2577aecc430bb741449e3d001781d84275baba8044bf2e21d16abf17ff43310081f5318ddd0     1
57f3b666b5267652a62ae6a825309e6327349a0bd70

In [ ]:
model = CatBoostRanker(loss_function='YetiRank', max_depth=2, iterations=500, verbose=1, eval_metric='MAP:top=7', early_stopping_rounds=15)
model.fit(tr_pool, eval_set=val_pool)

Groupwise loss function. OneHotMaxSize set to 10
0:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 217ms	remaining: 1m 48s
1:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 457ms	remaining: 1m 53s
2:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 725ms	remaining: 2m
3:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 975ms	remaining: 2m
4:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 1.21s	remaining: 1m 59s
5:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 1.44s	remaining: 1m 58s
6:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 1.7s	remaining: 1m 59s
7:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 1.97s	remaining: 2m
8:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 2.19s	remaining: 1m 59s
9:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 2.43s	remaining: 1m 59s
10:	learn: 0.3269220	test: 0.3231132	best: 0.3231132 (0)	total: 2.65s	remaining: 1m 57s
11:	le

In [46]:
tr_pairs

customer_id  \
0         25849efef8138760f1c721eaf6df0bc18db18e49cb4d40...   
2         ddeedff3398479eb3fdb4d503216683d34b7c6ab63b8b8...   
3         1c0e46ab2eae97ca7e290796ba78e04ea4463f2cbf1334...   
5         6a8ffaffb3385412a2f6846a54964ad414e010b9bcbc9b...   
6         ddccaecc57ec6476d51575181a7cf15dfe1f2ac838fac8...   
...                                                     ...   
23387838  480be587c54353978d487a52947993de3972b18b675d7d...   
23387839  17daac0bc577ebc91ef3be719c4351ad12296ed174c38f...   
23387840  bc22bcc51c6820452b10500b45667d9c931b027232d5e4...   
23387841  b22a86ae7190d2f272202b16f5881eabb94944f10d423d...   
23387843  4f25f5e2623fe3162c7178389ececd72af781d722f4c8b...   

                                               community_id  target  \
0         a2219c413f4567b73a1f9e61fc9251d67a80d59929ca22...     1.0   
2         dd7e34ee3464baffbccc9a45b1f21d7863b56a0ef48eaa...     0.0   
3         aab58cf59f2b66deded9d9b2bd4ba936c70c4122fa21c2...     0.0   
5         612f883c6d5d11d0e8647322a2e3ffa91ce6c556264e69...     1.0   
6         541f065a4a44536ba388a69fcca5351f1727c2484a5d60...     1.0   
...                                                     ...     ...   
23387838  7f04b2c388238a99d9638ce8ea448936027853ceb12518...     1.0   
23387839  a38dcfb8954fffc8eec7b2420dcee193dca7c8dfc47ed9...     1.0   
23387840  336ff8eb5d3d86e59893411b768e216ae13d373eccfd76...     1.0   
23387841  872892c975286011a5bdbf41ccbba8f1a1331a06097d23...     1.0   
23387843  5af49ad041e93dbbb255252ec60711b9b08ae4761bde25...     0.0   

                                                       pair  
0         25849efef8138760f1c721eaf6df0bc18db18e49cb4d40...  
2         ddeedff3398479eb3fdb4d503216683d34b7c6ab63b8b8...  
3         1c0e46ab2eae97ca7e290796ba78e04ea4463f2cbf1334...  
5         6a8ffaffb3385412a2f6846a54964ad414e010b9bcbc9b...  
6         ddccaecc57ec6476d51575181a7cf15dfe1f2ac838fac8...  
...                                                     ...  
23387838  480be587c54353978d487a52947993de3972b18b675d7d...  
23387839  17daac0bc577ebc91ef3be719c4351ad12296ed174c38f...  
23387840  bc22bcc51c6820452b10500b45667d9c931b027232d5e4...  
23387841  b22a86ae7190d2f272202b16f5881eabb94944f10d423d...  
23387843  4f25f5e2623fe3162c7178389ececd72af781d722f4c8b...  

[18669782 rows x 4 columns]